In [1]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

from pdf2image import convert_from_path
import os
import cv2
from PIL import Image

In [9]:
class Item:
    
    def __init__(self, pdf_path):
        self.list_jpg = self.pdf_to_img(pdf_path)
       
    def extract_basic(self):
        """
        Extract string with basic scan of each pages from pdf
        @print string
        @return none
        """
        final_txt=''
        for item_jpg in self.list_jpg:
            add_string = self.img_to_string(item_jpg)
            final_txt = final_txt+f'\n page_{self.list_jpg.index(item_jpg)+1}\n'+add_string
            
        self.txt = final_txt
        print(self.txt)

    def img_to_string(self, img_path):
        """
        Scan element to string
        @return string with all scan
        """
        pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'
        return pytesseract.image_to_string(Image.open(img_path))
    
    def pdf_to_img(self, pdf_path):
        """
        Convert and save all pdf pages in jpeg files
        @return list of jpeg files
        """
       
        pages = convert_from_path(pdf_path, 350, poppler_path = r'/usr/local/Cellar/poppler/21.09.0/bin' )
        
        i = 1
        name = os.path.basename(pdf_path).split('.')[0]
        tab_jpg = []
        
        for page in pages:
            image_name = name + str(i) + ".jpg"  
            page.save(image_name, "JPEG")
            i = i+1
            tab_jpg.append(image_name)
        
        return tab_jpg
    
    def extract_with_cv2(self):
        """
        Extract string items from element scaned by cv2
        @print string
        @return none
        """
        self.string_cv2 = ""
        for img_item in self.list_jpg:
            
            add_string = self.mark_region(img_item)
            self.string_cv2 = self.string_cv2 + f'\n page_{self.list_jpg.index(img_item)+1}\n' + add_string
               
        print(self.string_cv2)
        
    def mark_region(self,jpg_path):
        """
        define coordinates of string elements in page and extract strings
        @return strings
        """
        
        im = cv2.imread(jpg_path)

        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (9,9), 0)
        thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

        # Dilate to combine adjacent text contours
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
        dilate = cv2.dilate(thresh, kernel, iterations=4)

        # Find contours, highlight text areas, and extract ROIs
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        line_items_coordinates = []
        for c in cnts:
            area = cv2.contourArea(c)
            x,y,w,h = cv2.boundingRect(c)

            if y >= 600 and x <= 1000:
                if area > 10000:
                    image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                    line_items_coordinates.append([(x,y), (2200, y+h)])

            if y >= 2400 and x<= 2000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])

        textes = ""
        
        for i in range(len(line_items_coordinates)):
            # get co-ordinates to crop the image
            c = line_items_coordinates[i]

            # cropping image img = image[y0:y1, x0:x1]
            img = im[c[0][1]:c[1][1], c[0][0]:c[1][0]]    

            # convert the image to black and white for better OCR
            ret,thresh1 = cv2.threshold(img,120,255,cv2.THRESH_BINARY)

            # pytesseract image to string to get results
            text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))

            textes = textes + '\n' + text

        return textes
        

In [10]:
test = Item('facture.pdf')

In [11]:
test.extract_basic()


 page_1
Page 1/1

DARTY GRAND OUEST

M BEYLARD PIERRE

 

Cc
Service Comptabilité 8 32 SEN DE MARIE GALANTE
TSA n° 90 005 5 33800 BORDEAUX
rance
93145 - Bondy cedex
0 978 970 970 (prix d'un appel local) < | MBEYLARD PIERRE
3 32 SEN DE MARIE GALANTE
3 33300 BORDEAUX
JUSTIFICATIF DE VENTE (valant facture) ® | France
Le 04/03/2021
Votre commande 9160108143690 du 28/01/2021
nee , . ; Date Montant / Taux
Référence Qté Libellé délivrance Total HT TVA Total TTC
3857433 1 MAGIM EXPRESSO M105 INISSIA 01/02/2021 58,32 € 11,66 69,99 €
CREME 20,00 %
Disponibilité des pieces détachées : 5 ans
Dont éco-participation DEEE 0,02 € 0,00 0,02 €
20,00 %
4593510 1 © KCOOK RECHAUD HPV240 01/02/2021 77,49 € 15,50 92,99 €
Disponibilité des pieces détachées : 1 an 20,00 %
Dont éco-participation DEEE 0,06 € 0,01 0,07 €
20,00 %

Le contrat de confiance. Total facturé : 135,81 € 27,16 € 162,98 €

Dont éco-participation DEEE : 0,08 € 0,09 €

 

Montant réglé par : Carte Bleue

Solde a régler :

 

DARTY GRAND OUE

In [12]:
test.extract_with_cv2()


 page_1

DARTY GRAND OUEST - SNC au capital de 30 612 EUR - RCS NANTES B 339 403
Adresse : Pare Tertiaire de l'Eraudiére - 32, rue de Coulongé - 44315 NANTES Cedex
TVA intracommunautaire : FR 95 339 403 933

DARTY GRAND OUEST - SNC au capital de 30 612 EUR - RCS NANTES B 339 403
Adresse : Pare Tertiaire de l'Eraudiére - 32, rue de Coulongé - 44315 NANTES Cedex
TVA intracommunautaire : FR 95 339 403 933

Montant reéglé par : Carte Bleue
Solde a ré

Montant reéglé par : Carte Bleue
Solde a ré

Lecontratdeconfiance.  —_ | Total facturé : 135,81 €

Lecontratdeconfiance.  —_ | Total facturé : 135,81 €

— eee
| Total facturé : 135,81 €
| consopmiootooeee Swat

— eee
| Total facturé : 135,81 €
| consopmiootooeee Swat


Dont éco-participation DEEE 0,06 €

4593510 1 KCOOK RECHAUD HPV240 01/02/2021 77,49 €
Disponibilité des pieces détachées : 1 an

Dont éco-participation DEEE 0,02 €

Libellé ee TotalHT

Référence Qté |Libellé ee TotalHT

Référence Qté |Libellé ue Total HT Om"?
385

In [13]:
Item('pretlcl.pdf').extract_basic()


 page_1
15/09/2021 09:32 Courrier - Pierre Beylard - Outlook

Le résultat de votre simulation prét immobilier LCL (Capacité d'achat)

LCL Pr@t Immobilier <lcl@infos.Icl.fr>
Mer 15/09/2021 07:27

A: lvlagicpeter@hotmail.com <Ivlagicpeter@hotmail.com>

FINANCEZ VOS PROJETS IMMOBILIERS

 

Madame, Monsieur,

Nous vous remercions d'avoir utilisé le simulateur LCL pour l'étude de votre projet
immobilier. Nous avons le plaisir de vous adresser ci-dessous le résultat de votre
simulation.

Cordialement,
LCL

 

FAIRE UNE DEMANDE DE PRET

Résultat de votre simulation du 15/09/2021

Montant d'emprunt
362 362,72 €

Mensualite
| 1 530,00 €
| Apport* Capacité d'achat
10 000 € 372 362,72 €

“Votre apport ne couvre pas les frais estimés. Vous disposez de 372 362,72 € (capacité
d'achat hors frais) .

TAUX NOMINAL TAUX ANNUEL EFFECTIF GLOBAL
1,46 % (1)(3) 2,10 % (2)

Votre simulation a la loupe (TAEG, TAEA...)

Détails du projet

Montant du prét a 1,46 % (1) sur 25 ans 362 362,72 €
Apport 10 000 €
Cap

In [14]:
test = Item('pretlcl.pdf')

In [15]:
test.extract_basic()


 page_1
15/09/2021 09:32 Courrier - Pierre Beylard - Outlook

Le résultat de votre simulation prét immobilier LCL (Capacité d'achat)

LCL Pr@t Immobilier <lcl@infos.Icl.fr>
Mer 15/09/2021 07:27

A: lvlagicpeter@hotmail.com <Ivlagicpeter@hotmail.com>

FINANCEZ VOS PROJETS IMMOBILIERS

 

Madame, Monsieur,

Nous vous remercions d'avoir utilisé le simulateur LCL pour l'étude de votre projet
immobilier. Nous avons le plaisir de vous adresser ci-dessous le résultat de votre
simulation.

Cordialement,
LCL

 

FAIRE UNE DEMANDE DE PRET

Résultat de votre simulation du 15/09/2021

Montant d'emprunt
362 362,72 €

Mensualite
| 1 530,00 €
| Apport* Capacité d'achat
10 000 € 372 362,72 €

“Votre apport ne couvre pas les frais estimés. Vous disposez de 372 362,72 € (capacité
d'achat hors frais) .

TAUX NOMINAL TAUX ANNUEL EFFECTIF GLOBAL
1,46 % (1)(3) 2,10 % (2)

Votre simulation a la loupe (TAEG, TAEA...)

Détails du projet

Montant du prét a 1,46 % (1) sur 25 ans 362 362,72 €
Apport 10 000 €
Cap

In [16]:
def extract_item(string_item, len_item, texte):
    b = test.txt.find(string_item) + len(string_item)+1
    return texte[b : b + len_item]

In [17]:
print(extract_item("Montant d'emprunt", 10, test.txt))

362 362,72


In [18]:
print(extract_item("Mensualite", 10, test.txt))

| 1 530,00


In [20]:
print(extract_item("votre simulation du", 10, test.txt))

15/09/2021


In [25]:
test = Item('num_files/CMSO.pdf')

In [26]:
test.extract_with_cv2()


 page_1

Document non contractuel 1/5 Réf. ¢

Document non contractuel 1/5 Réf. ¢

145 Fé. {

E
3,
f
#

E
3,
f
#

Revenus actuels : 7 619,00 mois Remboursements de crédits : 1 446

Revenus actuels : 7 619,00 mois Remboursements de crédits : 1 446

Remboursements de crédits : 1 446 -

Nombre de personnes au foyer : 1

Situation farniliale : Céelibataire Nombre de personnes au foyer : 1

Situation farniliale : Céelibataire Nombre de personnes au foyer : 1

ee NE hein ee ee ee QE PS

SS eT Tete bee 5 Uae er me FS ee FS WE er nr |

I FN tee ee Slee Se SS te LE Fh OE A OO
33000 BORDEAUX

I FN tee ee Slee Se SS te LE Fh OE A OO
33000 BORDEAUX

me a sn ek
mye Tat ew TE PF ee ye WF 9 betwe hee SU Be Se ie Se te § Sle ey 5 af v SF SAE PR et WO

me a sn ek
mye Tat ew TE PF ee ye WF 9 betwe hee SU Be Se ie Se te § Sle ey 5 af v SF SAE PR et WO


7 Pte Eph ee ee a AG an G eae aed ad Pa
Pe TRULLI GE BO Pe BE EP RR Penne SANA EUS Mee ee 5 TT ORE Syste SS
oS etd Tor bt CoOL EEN Ee 

In [27]:
test.extract_basic()


 page_1
Rai: TA_MCRED1550002-1

CAISSE DE CREDIT MUTUEL

   

 

~ 33000 BORDEAUX

 

BORDEAUX, le 11 février 2020

 

Les informations ci-dessous ne sont pas contractuelles et ne peuvent tre assimilées ni & une offre de crédit ou
une proposition commerciale, ni a un engagement précontractuel de votre Caisse de Crédit Mutuel. Ces
informations sont uniquement données a titre informatif et indicatif, des informations, piéces et justificatifs étant
nécessaires ou obligatoires pour mener plus avant l’étude et fa faisabilifé de votre projet immobllier.

Rappel : paur le financement d'une opération relevant des articles 313-1 et suivants du code de la consommation,
/Emprunteur dispose d’un délai de réflexion de 10 jours aprés réception de offre de prét. La réalisation de la vente
est subordonnée a lobtention du prét. Si celul-ci n’est pas obtenu, le vendeur doit rembourser les sommes versées
(articles L. 313-3 du code de la consommation).

   

CAISSE DE CREDIT MUTUEL DE BORDEAUX SAINT AUGU

In [ ]:
test = Item('num_files/CMSO.pdf')